## Import tools

In [3]:
import numpy as np
import pandas as pd

## Get the data

In [4]:

def onehot_encoder(ary, columns=[], remove_trap=False):
    df_results = pd.DataFrame()

    # Iterate each column in DataFrame ary
    for i in range(ary.shape[1]):
        # if this column (i) is dummy column
        if i in columns:
            base_name = ary.columns[i]
            this_column = pd.get_dummies(ary.iloc[:, i])
            this_column = this_column.rename(columns={n:"{}_{}".format(base_name, n) for n in this_column.columns})
            # Remove Dummy Variable Trap if needed
            if remove_trap:
                this_column = this_column.drop(this_column.columns[0], axis=1)
        # else this column is normal column
        else:
            this_column = ary.iloc[:, i]
        # Append this column to the Result DataFrame
        df_results = pd.concat([df_results, this_column], axis=1)

    return df_results


## Node class

In [5]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

## Tree class

In [6]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Train-Test split

In [7]:
train_df = pd.read_csv('./train_preproceed.csv')
test_df = pd.read_csv('./test.csv')
train_df_ = onehot_encoder(train_df, columns=[1, 2], remove_trap=True)
#test_df = onehot_encoder(test_df, columns=[1, 2], remove_trap=True)
train_df = train_df_.sample(n=1000, random_state=42)  # 使用 random_state 以确保可重复性
X_train = train_df.iloc[:, :-1].values



Y_train = train_df.iloc[:, -1].values.reshape(-1,1)

val_df = train_df_.sample(n=1000, random_state=38)  # 使用 random_state 以确保可重复性


X_test = val_df.iloc[:, :-1].values
Y_test = val_df.iloc[:, -1].values.reshape(-1,1)
#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

## Fit the model

In [8]:
classifier = DecisionTreeClassifier(min_samples_split=6, max_depth=6)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_34 <= 0.107088525817622 ? 0.016547664590467304
 left:X_32 <= 0.0176580605827879 ? 0.09124227230999338
  left:X_0 <= 8.948752612200874 ? 0.04165289256198357
    left:X_33 <= 0.5239858988744482 ? 0.0786394557823129
        left:X_17 <= 0.0 ? 0.08185941043083911
                left:X_34 <= 0.010404099195886 ? 0.10208333333333341
                                left:0.0
                                right:X_33 <= 0.5124623763033758 ? 0.035555555555555465
                                                                left:1.0
                                                                right:1.0
                right:0.0
        right:X_15 <= 0.0 ? 0.12346938775510213
                left:X_21 <= 0.0 ? 0.1866666666666665
                                left:0.0
                                right:1.0
                right:1.0
    right:X_27 <= 0.0 ? 0.24500000000000002
        left:X_6 <= 0.0 ? 0.10208333333333341
                left:X_33 <= 0.576923076923077 ? 0.035555555555555

## Test the model

In [9]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.589

## Pruning via reduced node_number_per_root and depth of tree, computational time is reduced accoredingly, and without performance degradation

In [10]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_34 <= 0.107088525817622 ? 0.016547664590467304
 left:X_32 <= 0.0176580605827879 ? 0.09124227230999338
  left:X_0 <= 8.948752612200874 ? 0.04165289256198357
    left:X_33 <= 0.5239858988744482 ? 0.0786394557823129
        left:1.0
        right:0.0
    right:X_27 <= 0.0 ? 0.24500000000000002
        left:0.0
        right:1.0
  right:X_3 <= 0.0 ? 0.027901785714285685
    left:X_1 <= 0.0 ? 0.017185821697099812
        left:0.0
        right:0.0
    right:1.0
 right:X_33 <= 0.288461538461538 ? 0.008485516938030513
  left:X_0 <= 9.359923901489068 ? 0.03385416666666667
    left:0.0
    right:X_4 <= 0.0 ? 0.4444444444444444
        left:1.0
        right:0.0
  right:X_20 <= 0.0 ? 0.005492655707324101
    left:X_32 <= 0.1396491872578163 ? 0.010607895739317363
        left:1.0
        right:0.0
    right:X_33 <= 0.634615384615385 ? 0.008635395338997376
        left:1.0
        right:1.0


In [11]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.561